# Dappier

> [Dappier](https://dappier.com) connects any LLM or your Agentic AI to real-time, rights-cleared, proprietary data from trusted sources, making your AI an expert in anything. Our specialized models include Real-Time Web Search, News, Sports, Financial Stock Market Data, Crypto Data, and exclusive content from premium publishers. Explore a wide range of data models in our marketplace at [marketplace.dappier.com](https://marketplace.dappier.com).

> [Dappier](https://dappier.com) delivers enriched, prompt-ready, and contextually relevant data strings, optimized for seamless integration with LangChain. Whether you're building conversational AI, recommendation engines, or intelligent search, Dappier's LLM-agnostic RAG models ensure your AI has access to verified, up-to-date data—without the complexity of building and managing your own retrieval pipeline.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `dappier` package itself.

```bash
pip install -U langchain-community dappier
```

You also need to set our Dappier API credentials, which can be generated at the [Dappier site.](https://platform.dappier.com).

You can find the supported data models by heading over to the [Dappier marketplace.](https://platform.dappier.com/marketplace)

In [3]:
import getpass
import os

os.environ["DAPPIER_API_KEY"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

In [23]:
from langchain_community.retrievers import DappierRetriever

retriever = DappierRetriever(data_model_id="dm_01jagy9nqaeer9hxx8z1sk1jx6")

retriever.invoke("latest tech news")

[Document(metadata={'title': 'Hannah Hidalgo scores 29 as No. 8 Notre Dame holds off No. 2 UConn', 'author': 'Curt Rallo, Associated Press', 'source_url': 'https://www.wishtv.com/sports/college-basketball/hannah-hidalgo-scores-29-as-no-8-notre-dame-holds-off-no-2-uconn/', 'image_url': 'https://images.dappier.com/dm_01jagy9nqaeer9hxx8z1sk1jx6/ONLINE-CROP-Notre-Dame-Hannah-Hidalgo-AP24348035473873_.jpg?width=428&height=321', 'pubdata': 'Fri, 13 Dec 2024 03:03:56 +0000'}, page_content="Hannah Hidalgo led No. 8 Notre Dame to a 79-68 victory over No. 2 UConn, marking the Huskies' first loss of the season. Hidalgo scored 29 points, while Olivia Miles and Liatu King each added 16. Notre Dame's strong defense limited UConn to just 3-of-16 from three-point range, contrasting with their own 10-for-18 performance. A pivotal buzzer-beater by Hidalgo at the end of the third quarter helped regain momentum for the Fighting Irish.\n\nLiza Karlen made her return for Notre Dame after an ankle injury, wh

In [26]:
# you have full control on filtering by category, time, pagination, and even the search method you use.
from datetime import datetime, timedelta

start = (datetime.now() - timedelta(days=7)).timestamp()
end = datetime.now().timestamp()

retriever = DappierRetriever(
    data_model_id="dm_01jagy9nqaeer9hxx8z1sk1jx6",
    k=5,
    ref="techcrunch.com",
    num_articles_ref=2,
    search_algorithm="most_recent"
)

retriever.invoke("latest tech news")

[Document(metadata={'title': 'Hannah Hidalgo scores 29 as No. 8 Notre Dame holds off No. 2 UConn', 'author': 'Curt Rallo, Associated Press', 'source_url': 'https://www.wishtv.com/sports/college-basketball/hannah-hidalgo-scores-29-as-no-8-notre-dame-holds-off-no-2-uconn/', 'image_url': 'https://images.dappier.com/dm_01jagy9nqaeer9hxx8z1sk1jx6/ONLINE-CROP-Notre-Dame-Hannah-Hidalgo-AP24348035473873_.jpg?width=428&height=321', 'pubdata': 'Fri, 13 Dec 2024 03:03:56 +0000'}, page_content="Hannah Hidalgo led No. 8 Notre Dame to a 79-68 victory over No. 2 UConn, marking the Huskies' first loss of the season. Hidalgo scored 29 points, while Olivia Miles and Liatu King each added 16. Notre Dame's strong defense limited UConn to just 3-of-16 from three-point range, contrasting with their own 10-for-18 performance. A pivotal buzzer-beater by Hidalgo at the end of the third quarter helped regain momentum for the Fighting Irish.\n\nLiza Karlen made her return for Notre Dame after an ankle injury, wh

## Chaining

We can easily combine this retriever in to a chain.

In [16]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """The following news articles may come in handy for answering the question:

{context}

Question:

{question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview")
    | StrOutputParser()
)

In [28]:
chain.invoke({"question": "What are the key highlights and outcomes from the latest events covered in the article?"})

"The key highlights and outcomes from the events mentioned in the articles are as follows:\n\n1. Hannah Hidalgo's Performance in Notre Dame vs. UConn Basketball Game:\n   - Hannah Hidalgo scored 29 points leading Notre Dame to a 79-68 victory over UConn.\n   - This was UConn's first loss of the season.\n   - Olivia Miles and Liatu King each contributed 16 points to Notre Dame's win.\n   - Notre Dame displayed strong defense, limiting UConn to 3-of-16 from three-point range.\n   - Hidalgo scored a pivotal buzzer-beater at the end of the third quarter.\n   - Notre Dame's Liza Karlen returned from an ankle injury.\n   - UConn played without Azzi Fudd due to a knee sprain.\n   - Both teams have upcoming games, with Notre Dame playing against Eastern Michigan and UConn against Georgetown.\n\n2. Death of Winchester Firefighter Rick Sanders:\n   - Rick Sanders died while fighting a fire on Watson Street in Winchester, Indiana.\n   - Sanders suffered a medical emergency that led to cardiac arr